<div class="alert alert-info">
И так. Это первый серьезный этап для меня на стажировке. Попытка спарсить данные с сайта https://rabota1000.ru/

In [1]:
import requests
import fake_useragent
import pandas as pd
import time
import lxml.etree
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import bs4

In [2]:
url = 'https://rabota1000.ru/russia'

In [3]:
df_DS = pd.DataFrame({
    'Название вакансии':[],
    'Название компании':[],
    'Город':[],
    'Зарплата':[],
    'Опыт работы':[],
    'Описание':[],
    'Требования':[],
    'Тип занятости':[],
    'Ключевые навыки':[],
    'Источник':[],
})

In [4]:
with webdriver.Chrome() as browser:
    ua = fake_useragent.UserAgent()
    headers={"user-agent": ua.random}
    lenta = {}
    
    # Открываем сайт
    browser.get(url)
    browser.maximize_window()
    browser.delete_all_cookies()
    time.sleep(2)
    
    # ищем поле "Должность, вакансия"
    inp1 = browser.find_element(By.XPATH, '//*[@id="autocomplete-root-1"]/fieldset/label/input[2]')
    inp1.send_keys('Data Analyst')
    time.sleep(1)

    # "Найти работу"
    browser.find_element(By.XPATH, '/html/body/div[1]/header/div/div[1]/div[2]/form/button[2]').send_keys(Keys.ENTER)
    time.sleep(1)
    
    # скроллинг до конца
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match = False
    while(match == False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount == lenOfPage:
            match = True
    
    
    # Берем проскороленную станицу и пасим все что видим
    cont = browser.find_element(By.CLASS_NAME, 'vacancy-list')
    soup = BeautifulSoup(cont.get_attribute('innerHTML'), 'html.parser')
    vacancies = soup.find_all('div', attrs={'x-data':'vacancy'})
    lenta = {}
    for a in vacancies:
        lenta['Название вакансии'] = " ".join((a.find('div', class_='vacancy__content').find('h2', attrs={'class':'text-lg'}).text).split('\n')[1].split())
        lenta['Название компании'] = c = " ".join((a.find('div', class_='vacancy__content').find('div', attrs={'class':'mb-4'}).find('span', attrs={'class':'block'}).text).split('\n')[1].split())
        lenta['Зарплата'] =" ".join((" ".join((a.find('div', class_='vacancy__content').find('p', attrs={'class':'mb-4'}).text).split('\n'))).strip().split())
        lenta['Источник'] = b = a.find('div', class_='vacancy__content').select('a')[1]['href']
        lenta['Город'] = (a.find('div', class_='vacancy__content').find('div', attrs={'class':'mb-4'}).find_all('span')[1].text).split('\n')[1].split()[0]
        
        # Парсим уже сами ссылки на hh
        r_hh = requests.get(b, headers=headers)
        soup_pc = BeautifulSoup(r_hh.content, 'html.parser')
        dom = lxml.etree.HTML(str(soup_pc))
        
        try:
            type_expensive = dom.xpath('//*[@id="HH-React-Root"]/div/div[3]/div[1]/div/div/div/div/div/div[1]/div[1]/div/p[2]/span')[0].text
            if type_expensive != None:
                lenta['Тип занятости'] = type_expensive
            else:
                lenta['Тип занятости'] = 'Не указано'
        except:
            lenta['Тип занятости'] = 'Не указано'
            
        try:
            type_expensive = dom.xpath('//*[@id="HH-React-Root"]/div/div[3]/div[1]/div/div/div/div/div/div[1]/div[1]/div/p[2]')[0].text
            if type_expensive != None:
                lenta['Требования'] = type_expensive
            else:
                lenta['Требования'] = 'Не указано'
        except:
            lenta['Требования'] = 'Не указано'
            
        try:
            type_expensive = dom.xpath('//*[@id="HH-React-Root"]/div/div[3]/div[1]/div/div/div/div/div/div[1]/div[1]/div/p[1]/span')[0].text
            if type_expensive != None:
                lenta['Опыт работы'] = type_expensive
            else:
                lenta['Опыт работы'] = 'Не указано'
        except:
            lenta['Опыт работы'] = 'Не указано'
        
        try:    
            type_expensive = soup_pc.find('div', class_='vacancy-section').find_all('p')
            if type_expensive != None:
                lenta['Описание'] = type_expensive
            else:
                lenta['Описание'] = 'Не указано'
        except:
            lenta['Описание'] = 'Не указано'
            
        type_expensive = soup_pc.find('div', class_='bloko-tag-list')
        if type_expensive != None:
            lenta['Ключевые навыки'] = type_expensive
        else:
            lenta['Ключевые навыки'] = 'Не указано'
        
        df_DS.loc[len(df_DS)] = lenta

C:\Users\Евгений\AppData\Roaming\Python\Python310\site-packages\pandas\core\dtypes\cast.py:950: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
df_DS['Описание'] = df_DS['Описание'].dropna(axis=0, how="all").apply(lambda x: " ".join([element.get_text() for element in x if type(element) == bs4.element.Tag]) if type(x) == bs4.element.ResultSet else x)
df_DS['Ключевые навыки'] = df_DS['Ключевые навыки'].dropna(axis=0, how="all").apply(lambda x: " ".join([element.get_text() for element in x if type(element) == bs4.element.Tag]) if type(x) == bs4.element.ResultSet else x)

In [ ]:
display(
    df_DS.head(),
    df_DS.tail()
)

In [ ]:
df_DS.to_csv(r"C:\\Users\\Евгений\\Internship\\1_Stage\\my_data2.csv", sep=';')